In [ ]:
import os, json, time, pickle
import pandas as pd, numpy as np 
import matplotlib.pyplot as plt

os.chdir('/home/jovyan/work/')

m_modelname = 'mistral_'
m_indatadir12 = os.path.join('personas', 'mistraldata_llm_1_2')
m_outdatadir12 = os.path.join('personas', 'mistral_study_1_2')
m_indatadir3 = os.path.join('personas', 'mistraldata_llm_3')
m_outdatadir3 = os.path.join('personas', 'mistral_study_3')
m_indatadir4 = os.path.join('personas', 'mistraldata_llm_4')
m_outdatadir4 = os.path.join('personas', 'mistral_study_4')
m_resultsdir = os.path.join('personas', 'mistralresults')

q_modelname = 'qwen_'
q_indatadir12 = os.path.join('personas', 'qwendata_llm_1_2')
q_outdatadir12 = os.path.join('personas', 'qwen_study_1_2')
q_indatadir3 = os.path.join('personas', 'qwendata_llm_3')
q_outdatadir3 = os.path.join('personas', 'qwen_study_3')
q_indatadir4 = os.path.join('personas', 'qwendata_llm_4')
q_outdatadir4 = os.path.join('personas', 'qwen_study_4')
q_resultsdir = os.path.join('personas', 'qwenresults')

modelname = 'joint_'
resultsdir = os.path.join('personas', 'joint_results')

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

##### Sample Personas for Table 1 

In [ ]:
pd_personas = pd.read_pickle('personas/data_ext/pd_personas_cleaned.pkl')
pd_personas.head(3)

In [ ]:
for i, row in pd_personas.sample(5, random_state=1).iterrows():
    print(row['personaId'], '&' ,row['persona'],'\\\\')

##### Display Clusters for Appendix Table

In [ ]:
pd_clusters = pd.read_pickle('personas/data_ext/pd_clusters_augmented.pkl')

In [ ]:
for i, row in pd_clusters.head(3).iterrows():
    print(i, '&', ', '.join(row['cluster_terms']), '&', '; '.join(row['cluster_personas'][:3]))

In [ ]:
for i, row in pd_clusters.tail(3).iterrows():
    print(i, '&', ', '.join(row['cluster_terms']), '&', '; '.join(row['cluster_personas'][:3]))

##### Utility functions

In [ ]:
from sklearn.metrics import classification_report

def votes_from_runs_crowd(pd_):
    pd_results = pd.DataFrame()
    for c in range(len(pd_.columns)):
        sums = list(pd_.iloc[:,:(c+1)].sum(axis=1))
        pd_results[f'run_{c}'] = [1 if s>(c+1)/2 else 0 for s in sums]
    return pd_results

def votes_from_runs_individual(pd_):
    personas = [p.split('_')[1] for p in pd_.columns]
    pd_.columns = [f'run_{i}' for i in range(len(pd_.columns))]
    return pd_, personas

def reports_from_votes(pd_, groundtruth_label):
    results = []
    for c in range(len(pd_.columns)):
        results.append(classification_report(groundtruth_label, pd_[f'run_{c}'], output_dict=True))
    return results

def series_from_reports(list_, personas):
    if personas:
        pd_results = pd.DataFrame(personas, columns=['personaId'])
    else:
        pd_results = pd.DataFrame([i+1 for i in range(len(list_))], columns=['crowdsize'])
    for e in ['0', '1']:
        for k in list_[0][e].keys():
            pd_results[e+'_'+k] = [l[e][k] for l in list_]
    for k in list_[0]['macro avg'].keys():
        pd_results['mavg'+'_'+k] = [l['macro avg'][k] for l in list_]
    for k in list_[0]['weighted avg'].keys():
        pd_results['wavg'+'_'+k] = [l['weighted avg'][k] for l in list_]
    pd_results['accuracy'] = [l['accuracy'] for l in list_]
    return pd_results.drop(['0_support','1_support','mavg_support','wavg_support'], axis=1)

def plot_series(dict_results, savename):
    fig, axs = plt.subplots(5, 3, figsize=(6,8))
    for k_type, v_type in dict_results.items():
        r,c = 0,0
        for i in range(13):
            for k_run, v_run in v_type['dfs'].items():
                axs[r,c].plot(v_run.iloc[:,0], v_run.iloc[:,i+1], color=v_type['color'], label=v_type['label'], linewidth=0.5)
                axs[r,c].set_ylim((0,1.05))
                axs[r,c].title.set_text(v_run.columns[i+1])
            if c < 2:
                c += 1
            else:
                c = 0
                r += 1
    fig.delaxes(axs[4,1])
    fig.delaxes(axs[4,2])
    fig.tight_layout()
    plt.savefig(os.path.join(resultsdir, modelname+f'{savename}.pdf'))
    plt.show()

def plot_single_series(dict_results, metric, savename):
    fig, ax = plt.subplots(1, 1, figsize=(5,3))
    min_s, max_s = 1, 0
    for k_type, v_type in dict_results.items():
        for k_run, v_run in v_type['dfs'].items():
            ax.plot(v_run['crowdsize'], v_run[metric], color=v_type['color'], label=v_type['label'], linewidth=0.5)
            min_r, max_r = v_run[metric].min(), v_run[metric].max()
            min_s = min_r if min_r < min_s else min_s
            max_s = max_r if max_r > max_s else max_s
    ax.set_ylim((min_s-0.05,max_s+0.05))
    ax.set(xlabel='crowd size', ylabel=metric)
    plt.savefig(os.path.join(resultsdir, modelname+f'{savename}.pdf'))
    plt.show()

def plot_boxplots(dict_results, savename):
    metrics = [list(v.columns) for k,v in dict_results.items()][0][1:]
    fig, axs = plt.subplots(5, 3, figsize=(6,8))
    r, c = 0, 0
    for i in range(13):
        data_ = {k: v.iloc[:,1+i] for k,v in dict_results.items()}
        axs[r,c].boxplot(data_.values())
        # axs[r,c].set_xticklabels(['p_you','np_you','p_any','np_any'], rotation=45)
        axs[r,c].set_xticklabels(['p','np'], rotation=45)
        # axs[r,c].set_xticklabels(['p','np','p-old','np-old'], rotation=45)
        axs[r,c].set_ylim([0,1.05])
        axs[r,c].title.set_text(metrics[i])
        if c < 2:
            c += 1
        else:
            c = 0
            r += 1
    fig.delaxes(axs[4,1])
    fig.delaxes(axs[4,2])
    fig.tight_layout()
    plt.savefig(os.path.join(resultsdir, modelname+f'{savename}.pdf'))
    plt.show()

def plot_single_boxplot(dict_results, metric, savename):
    fig, ax = plt.subplots(1,1, figsize=(5,3))
    data_ = {k: v[metric] for k,v in dict_results.items()}
    min_b, max_b = 1, 0
    for v in data_.values():
        min_b = v.min() if v.min() < min_b else min_b
        max_b = v.max() if v.max() > max_b else max_b
    ax.boxplot(data_.values())
    ax.set_xticklabels(['p','np'])
    # ax.set_xticklabels(['p','np','p-old','np-old'], rotation=45)
    ax.set_ylim([min_b-0.05,max_b+0.05])
    ax.set(ylabel=metric)
    plt.savefig(os.path.join(resultsdir, modelname+f'{savename}.pdf'))
    plt.show()

import dataframe_image as dfi

def create_table(dict_results, shorts, savename):
    pd_scores = pd.DataFrame([list(v.columns) for k,v in dict_results.items()][0][1:], columns=['metric'])
    for t in ['min','mean','50%', 'max']:
        for k,v in dict_results.items():
            pd_scores[f'{shorts[k]}_{t}'] = list(v.describe().loc[t,:])
    pd_scores = np.round(pd_scores, 4)
    dfi.export(pd_scores, os.path.join(resultsdir, modelname+f'{savename}.png'), table_conversion='matplotlib')
    return pd_scores

### Study 1

#### Load Data

In [ ]:
pd_data = pd.read_pickle('personas/data_ext/lscale_majVote.pkl')

m_pd_persona = pd.read_pickle(os.path.join(m_indatadir, '1000_persona_you.pkl'))
m_pd_nopersona = pd.read_pickle(os.path.join(m_indatadir, '1000_nopersona_you.pkl'))

q_pd_persona = pd.read_pickle(os.path.join(q_indatadir, '1000_persona_you.pkl'))
q_pd_nopersona = pd.read_pickle(os.path.join(q_indatadir, '1000_nopersona_you.pkl'))

#### Figures Crowds

In [ ]:
groundtruth = pd_data['ogLabelToxic']
size_crowd = 100

m_raw_runs = {
    'persona': {i: m_pd_persona.iloc[:,3+i*size_crowd:3+(i+1)*size_crowd].replace({'FALSE': 0, 'TRUE': 1}) for i in range(int((len(m_pd_persona.columns) - 3) / size_crowd))},
    'nopersona': {i: m_pd_nopersona.iloc[:,3+i*size_crowd:3+(i+1)*size_crowd].replace({'FALSE': 0, 'TRUE': 1}) for i in range(int((len(m_pd_nopersona.columns) - 3) / size_crowd))},
}

q_raw_runs = {
    'persona': {i: q_pd_persona.iloc[:,3+i*size_crowd:3+(i+1)*size_crowd].replace({'FALSE': 0, 'TRUE': 1}) for i in range(int((len(q_pd_persona.columns) - 3) / size_crowd))},
    'nopersona': {i: q_pd_nopersona.iloc[:,3+i*size_crowd:3+(i+1)*size_crowd].replace({'FALSE': 0, 'TRUE': 1}) for i in range(int((len(q_pd_nopersona.columns) - 3) / size_crowd))},
}

In [ ]:
m_results_runs_series = {}
m_dict_results_boxplots = {}

for k_type, v_type in m_raw_runs.items():
    m_results_runs_series[k_type] = {}
    pd_boxplots = pd.DataFrame()
    for k_run, v_run in v_type.items():
        votes = votes_from_runs_crowd(v_run)
        reports = reports_from_votes(votes, groundtruth)
        m_results_runs_series[k_type][k_run] = series_from_reports(reports, None)
        pd_boxplots = pd.concat([pd_boxplots, v_run], axis=1)
    pd_boxplots, personas = votes_from_runs_individual(pd_boxplots)
    reports = reports_from_votes(pd_boxplots, groundtruth)
    m_dict_results_boxplots[k_type] = series_from_reports(reports, personas)

In [ ]:
q_results_runs_series = {}
q_dict_results_boxplots = {}

for k_type, v_type in q_raw_runs.items():
    q_results_runs_series[k_type] = {}
    pd_boxplots = pd.DataFrame()
    for k_run, v_run in v_type.items():
        votes = votes_from_runs_crowd(v_run)
        reports = reports_from_votes(votes, groundtruth)
        q_results_runs_series[k_type][k_run] = series_from_reports(reports, None)
        pd_boxplots = pd.concat([pd_boxplots, v_run], axis=1)
    pd_boxplots, personas = votes_from_runs_individual(pd_boxplots)
    reports = reports_from_votes(pd_boxplots, groundtruth)
    q_dict_results_boxplots[k_type] = series_from_reports(reports, personas)

In [ ]:
m_dict_results_series = {
    'persona': {'dfs': m_results_runs_series['persona'], 'color': 'blue', 'label': 'persona'},
    'nopersona': {'dfs': m_results_runs_series['nopersona'], 'color': 'red', 'label': 'no persona you'},
}

In [ ]:
q_dict_results_series = {
    'persona': {'dfs': q_results_runs_series['persona'], 'color': 'blue', 'label': 'persona'},
    'nopersona': {'dfs': q_results_runs_series['nopersona'], 'color': 'red', 'label': 'no persona you'},
}

In [ ]:
def plot_joint_series(m_dict_results, q_dict_results, metric, savename):
    fig, axs = plt.subplots(1, 2, figsize=(7,3))
    min_s, max_s = 1, 0
    
    for k_type, v_type in m_dict_results.items():
        for k_run, v_run in v_type['dfs'].items():
            axs[0].plot(v_run['crowdsize'], v_run[metric], color=v_type['color'], label=v_type['label'], linewidth=0.5)
            min_r, max_r = v_run[metric].min(), v_run[metric].max()
            min_s = min_r if min_r < min_s else min_s
            max_s = max_r if max_r > max_s else max_s

    for k_type, v_type in q_dict_results.items():
        for k_run, v_run in v_type['dfs'].items():
            axs[1].plot(v_run['crowdsize'], v_run[metric], color=v_type['color'], label=v_type['label'], linewidth=0.5)
            min_r, max_r = v_run[metric].min(), v_run[metric].max()
            min_s = min_r if min_r < min_s else min_s
            max_s = max_r if max_r > max_s else max_s

    axs[0].set_title('Mistral')
    axs[1].set_title('Qwen')
    axs[0].set_ylim((min_s-0.05,max_s+0.05))
    axs[1].set_ylim((min_s-0.05,max_s+0.05))
    axs[0].set(xlabel='crowd size', ylabel=metric)
    axs[1].set(xlabel='crowd size') #, ylabel=metric)
    plt.savefig(os.path.join(resultsdir, modelname+f'{savename}.pdf'), bbox_inches='tight')
    plt.show()

In [ ]:
plot_joint_series(m_dict_results_series, q_dict_results_series, 'mavg_f1-score', 'study_1_crowds_mavg_f1-score')

#### Figures Permutations

In [ ]:
with open(os.path.join(m_outdatadir,'crowd_permutations.pkl'), 'rb') as f:
    m_dict_permutations_series = pickle.load(f)

In [ ]:
with open(os.path.join(q_outdatadir,'crowd_permutations.pkl'), 'rb') as f:
    q_dict_permutations_series = pickle.load(f)

In [ ]:
plot_joint_series(m_dict_permutations_series, q_dict_permutations_series, 'mavg_f1-score', 'study_1_permutations')

### Study 2

#### Boxplots

In [ ]:
with open(os.path.join('personas','joint_results','mistral_stability.pkl'), 'rb') as f:
    mistral_dicts = pickle.load(f)

with open(os.path.join('personas','joint_results','qwen_stability.pkl'), 'rb') as f:
    qwen_dicts = pickle.load(f)

In [ ]:
m_data_ = mistral_dicts['data_']
m_bracket_boxplots = mistral_dicts['bracket_boxplots']
m_brackets = mistral_dicts['brackets']

In [ ]:
q_data_ = qwen_dicts['data_']
q_bracket_boxplots = qwen_dicts['bracket_boxplots']
q_brackets = qwen_dicts['brackets']

##### statistical test: equality of variance in two non-normal samples (levene test)
h0: all input samples from populations with equal variances

In [ ]:
from scipy import stats

In [ ]:
stats.levene(m_data_['persona'], m_data_['nopersona'])

In [ ]:
stats.levene(q_data_['persona'], q_data_['nopersona'])

##### figures

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(11,8),gridspec_kw={'width_ratios': [1, 5]})

axs[0,0].boxplot(m_data_.values(), widths=0.7)
# axs[0,0].set_xticklabels(['pers.','no pers.']) #, rotation=45)
axs[0,0].set_xticks([])
# axs[0,0].set_ylabel('mavg_f1-score')
axs[0,0].set_ylabel('Mistral')
axs[0,0].set_ylim((0.35,0.85))
axs[0,0].set_title('a)')

axs[0,1].boxplot(m_bracket_boxplots.values())
axs[0,1].set_xlim((-2,92)) # create whitespace left and right
axs[0,1].set_ylim((0.35,0.85))
axs[0,1].plot([i+1 for i in range(len(m_brackets))], m_brackets['mavg_f1-score'], linestyle='', marker='o', markersize=2, mec='red')
# axs[0,1].set_xticks([15,45,75], ['30 min personas', '30 median persona', '30 max personas'])
axs[0,1].set_xticks([])
axs[0,1].set_yticks([])
axs[0,1].set_title('b)')
axs[0,1].plot((30.5,30.5),(0.35,0.85), color='black', ls='--', linewidth=0.4)
axs[0,1].plot((60.5,60.5),(0.35,0.85), color='black', ls='--', linewidth=0.4)

axs[1,0].boxplot(q_data_.values(), widths=0.7)
axs[1,0].set_xticklabels(['pers.','no pers.']) #, rotation=45)
# axs[1,0].set_ylabel('mavg_f1-score')
axs[1,0].set_ylabel('Qwen')
axs[1,0].set_ylim((0.35,0.85))
axs[1,0].set_title('c)')

axs[1,1].boxplot(q_bracket_boxplots.values())
axs[1,1].set_xlim((-2,92)) # create whitespace left and right
axs[1,1].set_ylim((0.35,0.85))
axs[1,1].plot([i+1 for i in range(len(q_brackets))], q_brackets['mavg_f1-score'], linestyle='', marker='o', markersize=2, mec='red')
axs[1,1].set_xticks([15,45,75], ['30 min personas', '30 median persona', '30 max personas'])
axs[1,1].set_yticks([])
axs[1,1].set_title('d)')
axs[1,1].plot((30.5,30.5),(0.35,0.85), color='black', ls='--', linewidth=0.4)
axs[1,1].plot((60.5,60.5),(0.35,0.85), color='black', ls='--', linewidth=0.4)

plt.subplots_adjust(wspace=0.02, hspace=0.1)
plt.savefig(os.path.join('personas','joint_results', modelname+'study_2_stability.pdf'), bbox_inches='tight')
plt.show()

##### get persona descriptions and write to .xlsx for qualitative analysis

In [ ]:
pd_personas = pd.read_pickle(os.path.join('personas','data_ext','pd_personas_cleaned.pkl'))

In [ ]:
m_brackets = m_brackets.astype({'personaId': int})
q_brackets = q_brackets.astype({'personaId': int})

In [ ]:
m_brackets = m_brackets.merge(pd_personas[['personaId','persona']], on='personaId', how='left')
q_brackets = q_brackets.merge(pd_personas[['personaId','persona']], on='personaId', how='left')

In [ ]:
m_brackets.to_excel(os.path.join('personas','joint_results','bracket_personas_mistral.xlsx'))
q_brackets.to_excel(os.path.join('personas','joint_results','bracket_personas_qwen.xlsx'))

### Study 3

#### Normalized label-space distances for persona-space clusters 

In [ ]:
with open(os.path.join('personas','mistral_study_3','all_dists_full.pkl'), 'rb') as f:
    m_all_dists_full = pickle.load(f)

m_all_dists_personas_full = m_all_dists_full['personas']
m_all_dists_labels_full = m_all_dists_full['labels']

In [ ]:
with open(os.path.join('personas','qwen_study_3','all_dists_full.pkl'), 'rb') as f:
    q_all_dists_full = pickle.load(f)

q_all_dists_personas_full = q_all_dists_full['personas']
q_all_dists_labels_full = q_all_dists_full['labels']

In [ ]:
m_data = m_all_dists_labels_full.astype(float).copy()
q_data = q_all_dists_labels_full.astype(float).copy()

In [ ]:
m_norm_data = (m_data - np.min(m_data, axis=1)) / np.ptp(m_data, axis=1)
q_norm_data = (q_data - np.min(q_data, axis=1)) / np.ptp(q_data, axis=1)

In [ ]:
# m_argmins = []
# for row in range(m_data.shape[0]):
#     m_argmins.append(np.argmin(m_data[row,:]))

In [ ]:
# q_argmins = []
# for row in range(q_data.shape[0]):
#     q_argmins.append(np.argmin(q_data[row,:]))

In [ ]:
print(m_norm_data.shape)
print(q_norm_data.shape)

##### full versions - separately for zoom plot

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(5,5))

im1 = ax1.imshow(m_norm_data, interpolation='nearest', cmap='viridis_r', origin='lower')
ax1.set_xlabel('Persona Cluster ID X')
ax1.set_ylabel('Persona Cluster ID Y')
ax1.set_yticks([0,500,1000,1500,2000])
ax1.set_yticklabels([0,500,1000,1500,2000])
ax1.set_xticks([0,500,1000,1500,2000])
ax1.set_xticklabels([0,500,1000,1500,2000])
ax1.set_title('Mistral')

plt.savefig(os.path.join(resultsdir,modelname+'study_2_pcluster_distances_label_normalized_mistral.png'), bbox_inches='tight')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(5,5))

im1 = ax1.imshow(q_norm_data, interpolation='nearest', cmap='viridis_r', origin='lower')
ax1.set_xlabel('Persona Cluster ID X')
ax1.set_ylabel('Persona Cluster ID Y')
ax1.set_yticks([0,500,1000,1500,2000])
ax1.set_yticklabels([0,500,1000,1500,2000])
ax1.set_xticks([0,500,1000,1500,2000])
ax1.set_xticklabels([0,500,1000,1500,2000])
ax1.set_title('Qwen')

plt.savefig(os.path.join(resultsdir,modelname+'study_2_pcluster_distances_label_normalized_qwen.png'), bbox_inches='tight')
plt.show()

##### zoomed versions - separately for zoom plot

In [ ]:
cmin, cmax = 1000, 1100
m_norm_zoom = m_norm_data[cmin:cmax,cmin:cmax]
q_norm_zoom = q_norm_data[cmin:cmax,cmin:cmax]

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(5,5))

im1 = ax1.imshow(m_norm_zoom, interpolation='nearest', cmap='viridis_r', origin='lower')
ax1.set_yticks([])
ax1.set_yticklabels([])
ax1.set_xticks([20*(i+1) for i in range(4)])
ax1.set_xticklabels([cmin+20*(i+1) for i in range(4)])

plt.savefig(os.path.join(resultsdir,modelname+'study_2_pcluster_distances_label_zoomed_mistral.png'), bbox_inches='tight')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(5,5))

im1 = ax1.imshow(q_norm_zoom, interpolation='nearest', cmap='viridis_r', origin='lower')
ax1.set_yticks([])
ax1.set_yticklabels([])
ax1.set_xticks([20*(i+1) for i in range(4)])
ax1.set_xticklabels([cmin+20*(i+1) for i in range(4)])

plt.savefig(os.path.join(resultsdir,modelname+'study_2_pcluster_distances_label_zoomed_qwen.png'), bbox_inches='tight')
plt.show()

#### Normalized persona-space distances for label-space clusters 

In [ ]:
import torch

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

def distance_between_clusters(vectors_i, vectors_j):
    dists = cosine_distances(vectors_i, vectors_j)**2
    return np.mean(dists)

def create_distance_matrix(dict_clusters):
    cluster_distances = []
    for k1, v1 in dict_clusters.items():
        dists = []
        for k2, v2 in dict_clusters.items():
            if k1 > k2:
                dists.append(None)
            else:
                dists.append(distance_between_clusters(v1, v2))
        cluster_distances.append(dists)
    return cluster_distances

def create_full_matrix(mx):
    new_mx = np.matrix(mx)
    for i in range(new_mx.shape[0]):
        for j in range(new_mx.shape[1]):
            if i > j:
                new_mx[i,j] = mx[j][i]
            else:
                new_mx[i,j] = mx[i][j]
    return new_mx

def calculate_cluster_distances(pd_clusters_, persona_embeddings, label_embeddings, lclusters=False):
    if not lclusters:
        cluster_personas = {i: persona_embeddings[pd_clusters_.iloc[i,2]] for i in range(len(pd_clusters_))}
        cluster_labels = {i: label_embeddings[pd_clusters_.iloc[i,2]] for i in range(len(pd_clusters_))}
    else:
        cluster_personas = {i: persona_embeddings[list(pd_clusters_[pd_clusters_['clusterId_label']==i]['personaIndex'])] for i in range(pd_clusters_['clusterId_label'].max()+1)}
        cluster_labels = {i: label_embeddings[list(pd_clusters_[pd_clusters_['clusterId_label']==i]['personaIndex'])] for i in range(pd_clusters_['clusterId_label'].max()+1)}
    
    cluster_distances_personas = create_distance_matrix(cluster_personas)
    cluster_distances_labels = create_distance_matrix(cluster_labels)

    cluster_distances_personas_full = create_full_matrix(cluster_distances_personas)
    cluster_distances_labels_full = create_full_matrix(cluster_distances_labels)

    return cluster_distances_personas_full, cluster_distances_labels_full

mistral

In [ ]:
pd_lclusters = pd.read_pickle(os.path.join('personas','mistral_study_3','pd_lclusters.pkl'))

pd_personas_lclusters = pd.read_pickle(os.path.join('personas','mistral_study_3','pd_personas_lclusters.pkl')) # label-cluster membership of personas
pd_personas_lclusters['personaIndex'] = pd_personas_lclusters.index 

pd_personas_labels = pd.read_pickle(os.path.join('personas','mistral_study_3','pd_personas_labels.pkl')) # personas with persona and label embeddings

In [ ]:
lc_label_embeddings = torch.tensor(pd_personas_labels['labels']).float().cpu()
lc_persona_embeddings = torch.tensor(pd_personas_labels['embed']).float().cpu()

In [ ]:
%time m_lc_all_dists_personas_full, m_lc_all_dists_labels_full = calculate_cluster_distances(pd_personas_lclusters, lc_persona_embeddings, lc_label_embeddings, lclusters=True)

qwen

In [ ]:
pd_lclusters = pd.read_pickle(os.path.join('personas','qwen_study_3','pd_lclusters.pkl'))

pd_personas_lclusters = pd.read_pickle(os.path.join('personas','qwen_study_3','pd_personas_lclusters.pkl')) # label-cluster membership of personas
pd_personas_lclusters['personaIndex'] = pd_personas_lclusters.index 

pd_personas_labels = pd.read_pickle(os.path.join('personas','qwen_study_3','pd_personas_labels.pkl')) # personas with persona and label embeddings

In [ ]:
lc_label_embeddings = torch.tensor(pd_personas_labels['labels']).float().cpu()
lc_persona_embeddings = torch.tensor(pd_personas_labels['embed']).float().cpu()

In [ ]:
%time q_lc_all_dists_personas_full, q_lc_all_dists_labels_full = calculate_cluster_distances(pd_personas_lclusters, lc_persona_embeddings, lc_label_embeddings, lclusters=True)

combine

In [ ]:
m_data = m_lc_all_dists_personas_full.astype(float).copy()
q_data = q_lc_all_dists_personas_full.astype(float).copy()

In [ ]:
m_norm_data = (m_data - np.min(m_data, axis=1)) / np.ptp(m_data, axis=1)
q_norm_data = (q_data - np.min(q_data, axis=1)) / np.ptp(q_data, axis=1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

im1 = ax1.imshow(m_norm_data, interpolation='nearest', cmap='viridis_r', origin='lower')
ax1.set_xlabel('Persona Cluster ID X')
ax1.set_ylabel('Persona Cluster ID Y')
ax1.set_yticks([0,2,4,6,8,10])
ax1.set_yticklabels([0,2,4,6,8,10])
ax1.set_title('Mistral')
# ax1.scatter(m_argmins, [i for i in range(len(m_argmins))], color='red', s=.1)

im2 = ax2.imshow(q_norm_data, interpolation='nearest', cmap='viridis_r', origin='lower')
ax2.set_xlabel('Persona Cluster ID X')
ax2.set_yticks([])
ax2.set_title('Qwen')
# ax2.scatter(q_argmins, [i for i in range(len(q_argmins))], color='red', s=.1)

plt.subplots_adjust(wspace=0.07)
plt.savefig(os.path.join(resultsdir,modelname+'study_2_lcluster_distances_persona_normalized.pdf'), bbox_inches='tight')
plt.show()

##### plot persona space distances

In [ ]:
with open(os.path.join('personas','mistral_study_3','all_dists_full.pkl'), 'rb') as f:
    all_dists_full = pickle.load(f)

all_dists_personas_full = all_dists_full['personas']
all_dists_labels_full = all_dists_full['labels']

In [ ]:
data = all_dists_personas_full.astype(float).copy()

In [ ]:
norm_data = (data - np.min(data, axis=1)) / np.ptp(data, axis=1)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(5,5))

im1 = ax1.imshow(norm_data, interpolation='nearest', cmap='viridis_r', origin='lower')
ax1.set_xlabel('Persona Cluster ID X')
ax1.set_ylabel('Persona Cluster ID Y')
ax1.set_yticks([0,500,1000,1500,2000])
ax1.set_yticklabels([0,500,1000,1500,2000])
ax1.set_xticks([0,500,1000,1500,2000])
ax1.set_xticklabels([0,500,1000,1500,2000])

ax1.set_title('Both Models')
# ax1.scatter(m_argmins, [i for i in range(len(m_argmins))], color='red', s=.1)

# im2 = ax2.imshow(norm_data, interpolation='nearest', cmap='viridis_r', origin='lower')
# ax2.set_xlabel('Persona Cluster ID X')
# ax2.set_yticks([])
# ax2.set_title('Qwen')
# ax2.scatter(q_argmins, [i for i in range(len(q_argmins))], color='red', s=.1)

plt.subplots_adjust(wspace=0.07)
plt.savefig(os.path.join(resultsdir,modelname+'study_2_pcluster_distances_persona_normalized.png'), bbox_inches='tight')
plt.show()

#### Correlation Coefficients

In [ ]:
m_pd_spearman = pd.read_pickle(os.path.join('personas','mistral_study_3','correlations_spearman.pkl'))
q_pd_spearman = pd.read_pickle(os.path.join('personas','qwen_study_3','correlations_spearman.pkl'))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10,5))

axs[0].hist(m_pd_spearman[m_pd_spearman['pvalue']<0.05]['statistic'], bins=[-0.25+i*0.01 for i in range(50)])
axs[0].plot((0,0),(0,19000), color='red', ls='--', linewidth=0.8)
axs[0].set_ylim((0,19000))
axs[0].set_title('Mistral')

axs[1].hist(q_pd_spearman[q_pd_spearman['pvalue']<0.05]['statistic'], bins=[-0.25+i*0.01 for i in range(50)])
axs[1].set_yticks([])
axs[1].plot((0,0),(0,19000), color='red', ls='--', linewidth=0.8)
axs[1].set_ylim((0,19000))
axs[1].set_title('Qwen')

plt.subplots_adjust(wspace=0.07)
plt.savefig(os.path.join(resultsdir,modelname+'study_3_correlations.pdf'), bbox_inches='tight')
plt.show()

### Study 4

In [ ]:
dict_results = {'Mistral': {'AAE': {}, 'Anti-Black': {}}, 'Qwen': {'AAE': {}, 'Anti-Black': {}}}

In [ ]:
for m in ['Mistral', 'Qwen']:
    # for p in ['neutral_black', 'neutral_neutral', 'neutral_white']:
    for p in ['neutral_black', 'neutral_neutral', 'neutral_conservative']:
    # for p in ['neutral_black', 'neutral_africanamerican', 'neutral_neutral', 'neutral_conservative']:
        if m == 'Mistral':
            pd_ = pd.read_pickle(os.path.join(m_indatadir4,f'{p}_annotations.pkl'))
        else:
            pd_ = pd.read_pickle(os.path.join(q_indatadir4,f'{p}_annotations.pkl'))
        dict_results[m]['AAE'][p] = pd_[(pd_['isAAE']==True)&((pd_['isAntiBlack']==False))]
        dict_results[m]['Anti-Black'][p] = pd_[(pd_['isAAE']==False)&((pd_['isAntiBlack']==True))]

In [ ]:
for m_k, m_v in dict_results.items():
    for t_k, t_v in m_v.items():
        for p_k, p_v in t_v.items():
            print(m_k, t_k, p_k, np.round(p_v.iloc[:,10:].astype(int).mean().mean(), 2))

In [ ]:
q_aae_b = dict_results['Qwen']['AAE']['neutral_black'].iloc[:,10:].astype(int).mean()
q_aae_c = dict_results['Qwen']['AAE']['neutral_conservative'].iloc[:,10:].astype(int).mean()
q_aae_n = dict_results['Qwen']['AAE']['neutral_neutral'].iloc[:,10:].astype(int).mean()

m_aae_b = dict_results['Mistral']['AAE']['neutral_black'].iloc[:,10:].astype(int).mean()
m_aae_c = dict_results['Mistral']['AAE']['neutral_conservative'].iloc[:,10:].astype(int).mean()
m_aae_n = dict_results['Mistral']['AAE']['neutral_neutral'].iloc[:,10:].astype(int).mean()

q_ab_b = dict_results['Qwen']['Anti-Black']['neutral_black'].iloc[:,10:].astype(int).mean()
q_ab_c = dict_results['Qwen']['Anti-Black']['neutral_conservative'].iloc[:,10:].astype(int).mean()
q_ab_n = dict_results['Qwen']['Anti-Black']['neutral_neutral'].iloc[:,10:].astype(int).mean()

m_ab_b = dict_results['Mistral']['Anti-Black']['neutral_black'].iloc[:,10:].astype(int).mean()
m_ab_c = dict_results['Mistral']['Anti-Black']['neutral_conservative'].iloc[:,10:].astype(int).mean()
m_ab_n = dict_results['Mistral']['Anti-Black']['neutral_neutral'].iloc[:,10:].astype(int).mean()

In [ ]:
from scipy import stats

AAE: h0: black annotations >= neutral annotations; h1: black annotations < neutral annotations

In [ ]:
print(stats.ranksums(q_aae_b, q_aae_n, 'less'))
print(stats.ranksums(q_aae_c, q_aae_n, 'greater'))

In [ ]:
print(stats.ranksums(m_aae_b, m_aae_n, 'less'))
print(stats.ranksums(m_aae_c, m_aae_n, 'greater'))

In [ ]:
print(stats.ranksums(q_ab_b, q_ab_n, 'greater'))
print(stats.ranksums(q_ab_c, q_ab_n, 'less'))

In [ ]:
print(stats.ranksums(m_ab_b, m_ab_n, 'greater'))
print(stats.ranksums(m_ab_c, m_ab_n, 'less'))

#### Differences Per-Persona

In [ ]:
dict_diffs = {'Mistral': {'AAE': {}, 'Anti-Black': {}}, 'Qwen': {'AAE': {}, 'Anti-Black': {}}}

In [ ]:
for m in ['Mistral', 'Qwen']:
    for t in ['AAE', 'Anti-Black']:
        # persona_means_africanamerican = dict_results[m][t]['neutral_africanamerican'].iloc[:,10:].astype(int).mean()
        persona_means_black = dict_results[m][t]['neutral_black'].iloc[:,10:].astype(int).mean()
        persona_means_neutral = dict_results[m][t]['neutral_neutral'].iloc[:,10:].astype(int).mean()
        persona_means_conservative = dict_results[m][t]['neutral_conservative'].iloc[:,10:].astype(int).mean()
        # persona_means_white = dict_results[m][t]['neutral_white'].iloc[:,10:].astype(int).mean()
        
        # dict_diffs[m][t]['africanamerican'] = persona_means_africanamerican - persona_means_neutral
        dict_diffs[m][t]['black'] = persona_means_black - persona_means_neutral
        dict_diffs[m][t]['conservative'] = persona_means_conservative - persona_means_neutral
        # dict_diffs[m][t]['white'] = persona_means_white - persona_means_neutral

In [ ]:
dict_extremes = {
    'Mistral': {'min': 0, 'max': 0},
    'Qwen': {'min': 0, 'max': 0}
}

for m_k, m_v in dict_diffs.items():
    for t_k, t_v in m_v.items():
        for p_k, p_v in t_v.items():
            m_min, m_max = p_v.min(), p_v.max()
            if m_min < dict_extremes[m_k]['min']:
                dict_extremes[m_k]['min'] = m_min
            if m_max > dict_extremes[m_k]['max']:
                dict_extremes[m_k]['max'] = m_max

In [ ]:
dict_diffs_mistral = {}
for k,v in dict_diffs['Mistral']['AAE'].items():
    dict_diffs_mistral['Mistral_AAE_'+k] = v
for k,v in dict_diffs['Mistral']['Anti-Black'].items():
    dict_diffs_mistral['Mistral_Anti-Black_'+k] = v

dict_diffs_qwen = {}
for k,v in dict_diffs['Qwen']['AAE'].items():
    dict_diffs_qwen['Qwen_AAE_'+k] = v
for k,v in dict_diffs['Qwen']['Anti-Black'].items():
    dict_diffs_qwen['Qwen_Anti-Black_'+k] = v

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(9,3))

m_min, m_max = dict_extremes['Mistral']['min']*1.1, dict_extremes['Mistral']['max']*1.1
axs[0].boxplot(dict_diffs_mistral.values())
axs[0].set_ylim((m_min, m_max))
axs[0].set_title('Mistral')
# axs[0].set_xticks([1,2,3,4,5,6])
# axs[0].set_xticklabels(['afr.-am.','black','cons.','afr.-am.','black','cons'])
axs[0].set_xticks([1,2,3,4])
axs[0].set_xticklabels(['black','cons.','black','cons.'])
# axs[0].set_xticklabels(['black','white','black','white'])
axs[0].plot((2.5,2.5),(m_min, m_max), color='black', linewidth=0.4)
axs[0].plot((0.5,4.5),(0,0), color='blue', ls='--', linewidth=0.4)
axs[0].text(0.65,m_max*1.02,'AAE')
axs[0].text(3.5,m_max*1.02,'Anti-Black')

q_min, q_max = dict_extremes['Qwen']['min']*1.1, dict_extremes['Qwen']['max']*1.1
axs[1].boxplot(dict_diffs_qwen.values())
axs[1].set_ylim((q_min, q_max))
axs[1].set_title('Qwen')
# axs[1].set_xticks([1,2,3,4,5,6])
# axs[1].set_xticklabels(['afr.-am.','black','cons.','afr.-am.','black','cons'])
axs[1].set_xticks([1,2,3,4])
axs[1].set_xticklabels(['black','cons.','black','cons.'])
# axs[1].set_xticklabels(['black','white','black','white'])
axs[1].plot((2.5,2.5),(q_min, q_max), color='black', linewidth=0.4)
axs[1].plot((0.5,4.5),(0,0), color='blue', ls='--', linewidth=0.4)
axs[1].text(0.65,q_max*1.02,'AAE')
axs[1].text(3.5,q_max*1.02,'Anti-Black')

# plt.subplots_adjust(wspace=0.07)
plt.savefig(os.path.join(resultsdir,modelname+'study_4_diff_boxplots.pdf'), bbox_inches='tight')
plt.show()

#### Differences Per-Instance

In [ ]:
dict_diffs_instance = {'Mistral': {'AAE': {}, 'Anti-Black': {}}, 'Qwen': {'AAE': {}, 'Anti-Black': {}}}

In [ ]:
for m in ['Mistral', 'Qwen']:
    for t in ['AAE', 'Anti-Black']:
        instance_means_black = dict_results[m][t]['neutral_black'].iloc[:,10:].astype(int).mean(axis=1)
        instance_means_neutral = dict_results[m][t]['neutral_neutral'].iloc[:,10:].astype(int).mean(axis=1)
        instance_means_conservative = dict_results[m][t]['neutral_conservative'].iloc[:,10:].astype(int).mean(axis=1)

        pd_ = dict_results[m][t]['neutral_neutral'].iloc[:,:2]
        pd_['mean_black'] = instance_means_black
        pd_['mean_neutral'] = instance_means_neutral
        pd_['mean_conservative'] = instance_means_conservative
        pd_['black-conservative'] = instance_means_black - instance_means_conservative
        
        dict_diffs_instance[m][t] = pd_

##### Finding: Black Personas recognize the use of the n-word as a reclaimed slur, making these instance those with the biggest difference between black and conservative toxicity scores

QWEN

In [ ]:
dict_diffs_instance['Qwen']['AAE'].sort_values('black-conservative').head(3)

In [ ]:
dict_diffs_instance['Qwen']['AAE'].sort_values('black-conservative').tail(3)

In [ ]:
for i,row in dict_diffs_instance['Qwen']['AAE'].sort_values('black-conservative', ascending=True).head(3).iterrows():
    print(row['text'],'&',np.round(row['black-conservative'],2),'\\\\')

MISTRAL

In [ ]:
dict_diffs_instance['Mistral']['AAE'].sort_values('black-conservative').head(3)

In [ ]:
dict_diffs_instance['Mistral']['AAE'].sort_values('black-conservative').tail(3)

In [ ]:
for i,row in dict_diffs_instance['Mistral']['AAE'].sort_values('black-conservative', ascending=True).head(3).iterrows():
    print(row['text'],'&',np.round(row['black-conservative'],2),'\\\\')

##### Finding: Black Personas rate openly racist posts as much more toxic than the original "baseline" personas as well as their conservative counterparts do

QWEN

In [ ]:
dict_diffs_instance['Qwen']['Anti-Black'].sort_values('black-conservative', ascending=False).head(3)

In [ ]:
dict_diffs_instance['Qwen']['Anti-Black'].sort_values('black-conservative', ascending=False).tail(3)

In [ ]:
for i,row in dict_diffs_instance['Qwen']['Anti-Black'].sort_values('black-conservative', ascending=False).head(3).iterrows():
    print(row['text'],'&',np.round(row['black-conservative'],2),'\\\\')

In [ ]:
dict_diffs_instance['Mistral']['Anti-Black'].sort_values('black-conservative', ascending=False).head(3)

In [ ]:
dict_diffs_instance['Mistral']['Anti-Black'].sort_values('black-conservative', ascending=False).tail(3)

In [ ]:
for i,row in dict_diffs_instance['Mistral']['Anti-Black'].sort_values('black-conservative', ascending=False).head(3).iterrows():
    print(row['text'],'&',np.round(row['black-conservative'],2),'\\\\')